In [93]:
import numpy as np
import requests
import pandas as pd
import os

In [94]:
def make_df(resp):
    lst = []
    for day in resp.json():
        new_row = {
            "Province": day.get("provincia"),
            "Date": day.get("fecha"),
            "Wind": day.get("velmedia"),
            "Max_tmp": day.get("tmax"),
            "Sun": day.get("sol"),
            "Min_tmp": day.get("tmin"),
            "Mean_tmp": day.get("tmed"), "Rain": day.get("prec"), "presMax": day.get("presMax"),
            "presMin": day.get("presMin")}
        lst.append(new_row)

    df = pd.DataFrame(lst)
    return df

In [95]:
def process_df(df):
    df["Wind"] = df["Wind"].str.replace(',', '.').astype(float)
    df["Max_tmp"] = df["Max_tmp"].str.replace(',', '.').astype(float)
    df["Min_tmp"] = df["Min_tmp"].str.replace(',', '.').astype(float)
    df["Mean_tmp"] = df["Mean_tmp"].str.replace(',', '.').astype(float)
    df["presMax"] = df["presMax"].str.replace(',', '.').astype(float)
    df["presMin"] = df["presMin"].str.replace(',', '.').astype(float)
    df["Sun"] = df["Sun"].str.replace(',', '.').astype(float)

    df["Rain"] = df["Rain"].str.replace('Ip', '0')
    df["Rain"] = df["Rain"].str.replace(',', '.').astype(float)

    df["Date"] = pd.to_datetime(df["Date"])

    df = df.replace(to_replace=np.NaN, method='ffill')

    return df

In [ ]:
urls = {
    "Barcelona": "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2023-01-01T00:00:00UTC/fechafin/2023-10-31T23:59:59UTC/estacion/0076/?api_key=" + os.environ.get(
        "API_aemet"),
    "Madrid": "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2023-01-01T00:00:00UTC/fechafin/2023-10-31T23:59:59UTC/estacion/3129/?api_key=" + os.environ.get(
        "API_aemet"),
    "Sevilla": "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2023-01-01T00:00:00UTC/fechafin/2023-10-31T23:59:59UTC/estacion/5783/?api_key=" + os.environ.get(
        "API_aemet"),
    "Bilbao": "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2023-01-01T00:00:00UTC/fechafin/2023-10-31T23:59:59UTC/estacion/1082/?api_key=" + os.environ.get(
        "API_aemet"),
    "A coruna": "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2023-01-01T00:00:00UTC/fechafin/2023-10-31T23:59:59UTC/estacion/1387E/?api_key=" + os.environ.get(
        "API_aemet"),
    "Murcia": "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2023-01-01T00:00:00UTC/fechafin/2023-10-31T23:59:59UTC/estacion/7031X/?api_key=" + os.environ.get(
        "API_aemet"),
}

for name in urls:
    resp = requests.get(urls[name])
    resp = resp.json()
    url2 = resp["datos"]
    resp = requests.get(url2)
    df = make_df(resp)
    df = process_df(df)
    df.to_pickle("data/" + name + "_hist.pkl")
    df.to_csv("data/" + name + "_hist.csv")

# Combine DFs

In [97]:
# combine all dataframes

for name in urls:
    df = pd.read_pickle(name + "_hist.pkl")
    df = df[["Date", "Wind", "Max_tmp", "Min_tmp", "Mean_tmp", "Rain", "Sun", "presMax", "presMin"]]
    df["City"] = name
    if name == "Barcelona":
        df_wide = df
    else:
        df_wide = pd.concat([df_wide, df])

df_wide

,Date,Wind,Max_tmp,Min_tmp,Mean_tmp,Rain,Sun,presMax,presMin,City
0,2023-01-01,2.8,15.9,8.6,12.2,0.0,0.7,1026.8,1023.9,Barcelona
1,2023-01-02,3.3,15.0,7.0,11.0,0.0,0.2,1026.3,1022.8,Barcelona
2,2023-01-03,2.5,17.6,8.2,12.9,0.0,1.8,1032.0,1025.9,Barcelona
3,2023-01-04,2.8,16.1,7.5,11.8,0.0,5.4,1033.5,1031.2,Barcelona
4,2023-01-05,3.9,15.6,5.7,10.6,0.0,5.4,1031.5,1024.8,Barcelona
...,...,...,...,...,...,...,...,...,...,...
299,2023-10-27,2.8,25.0,13.0,19.0,0.0,2.9,1013.6,1008.3,Murcia
300,2023-10-28,3.1,23.7,10.0,16.8,0.0,10.4,1013.7,1010.0,Murcia
301,2023-10-29,3.6,24.6,9.5,17.0,0.1,8.6,1011.0,1006.8,Murcia
302,2023-10-30,5.0,23.3,14.6,19.0,0.0,7.9,1015.2,1005.4,Murcia


In [98]:
df_wide[df_wide["Date"] == "2023-01-01"]

,Date,Wind,Max_tmp,Min_tmp,Mean_tmp,Rain,Sun,presMax,presMin,City
0,2023-01-01,2.8,15.9,8.6,12.2,0.0,0.7,1026.8,1023.9,Barcelona
0,2023-01-01,1.7,16.4,3.9,10.2,0.1,3.3,958.5,952.6,Madrid
0,2023-01-01,1.4,16.4,6.7,11.6,6.0,2.4,1022.0,1018.2,Sevilla
0,2023-01-01,5.6,25.1,13.4,19.2,5.1,5.3,1014.9,1008.5,Bilbao
0,2023-01-01,3.6,13.9,8.1,11.0,15.1,0.0,1007.8,997.0,A coruna
0,2023-01-01,1.7,16.8,4.7,10.8,0.0,5.8,1027.0,1023.0,Murcia


In [99]:
# group by date and compute median values for columns
df_all = df_wide[["Date", "Wind", "Max_tmp", "Min_tmp", "Mean_tmp", "Rain", "Sun", "presMax", "presMin"]].groupby(
    "Date").mean()
df_all

,Wind,Max_tmp,Min_tmp,Mean_tmp,Rain,Sun,presMax,presMin
Date,,,,,,,,
2023-01-01,2.800000,17.416667,7.566667,12.500000,4.383333,2.916667,1009.500000,1003.866667
2023-01-02,2.366667,14.450000,6.916667,10.666667,0.816667,2.883333,1013.050000,1006.383333
2023-01-03,2.433333,15.683333,5.383333,10.533333,0.000000,6.033333,1017.916667,1012.700000
2023-01-04,2.266667,16.366667,6.016667,11.183333,0.000000,6.066667,1019.733333,1017.050000
2023-01-05,2.083333,15.900000,4.750000,10.316667,0.000000,7.416667,1017.350000,1011.383333
...,...,...,...,...,...,...,...,...
2023-10-27,4.850000,21.116667,12.466667,16.783333,0.450000,4.516667,996.283333,992.183333
2023-10-28,4.916667,21.150000,13.500000,17.316667,2.850000,5.533333,995.216667,990.600000
2023-10-29,4.400000,20.916667,12.816667,16.866667,5.066667,3.750000,992.350000,988.483333


# Get percents

In [100]:
# url = "https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2023-01-01T00:00&end_date=2023-10-31T23:59&time_trunc=day&geo_ids=8741&geo_trunc=electric_system&geo_limit=peninsular"
# resp = requests.get(url)
# resp = resp.json()

In [101]:
# lst = []
# for day in resp:
#     new_row = {"Province": day.get("provincia"), "Date": day.get("fecha"), "Max_tmp": day.get("tmax"),
#                "Sun": day.get("sol"),
#                "Min_tmp": day.get("tmin"),
#                "Mean_tmp": day.get("tmed"), "Rain": day.get("prec"), "presMax": day.get("presMax"),
#                "presMin": day.get("presMin")}
#     lst.append(new_row)
# 
# df = pd.DataFrame(lst)

In [102]:
df = pd.read_csv("energy_percentages.csv", sep=";")
df = df.rename(columns={"Fecha": "Date"})
df["Hidráulica"] = df["Hidráulica"].str.replace(',', '.').astype(float)
df["Turbinación bombeo"] = df["Turbinación bombeo"].str.replace(',', '.').astype(float)
df["Nuclear"] = df["Nuclear"].str.replace(',', '.').astype(float)
df["Carbón"] = df["Carbón"].str.replace(',', '.').astype(float)
df["Ciclo combinado"] = df["Ciclo combinado"].str.replace(',', '.').astype(float)
df["Eólica"] = df["Eólica"].str.replace(',', '.').astype(float)
df["Solar fotovoltaica"] = df["Solar fotovoltaica"].str.replace(',', '.').astype(float)
df["Solar térmica"] = df["Solar térmica"].str.replace(',', '.').astype(float)
df["Generación total"] = df["Generación total"].str.replace(',', '.').astype(float)

df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")
df2 = df[["Date", "Eólica", "Solar fotovoltaica", "Solar térmica", "Generación total"]]
df2["Solar"] = df["Solar fotovoltaica"] + df["Solar térmica"]
df2

C:\Users\tilen\AppData\Local\Temp\ipykernel_4136\4294534433.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Solar"] = df["Solar fotovoltaica"] + df["Solar térmica"]


,Date,Eólica,Solar fotovoltaica,Solar térmica,Generación total,Solar
0,2023-01-01,160.764699,33.392982,0.811405,524.323313,34.204387
1,2023-01-02,96.936318,30.595371,0.194716,632.047278,30.790087
2,2023-01-03,69.070871,52.667047,3.462572,667.943963,56.129619
3,2023-01-04,65.706889,56.871036,4.732083,632.489265,61.603119
4,2023-01-05,43.799077,56.868750,5.249039,635.988081,62.117789
...,...,...,...,...,...,...
299,2023-10-27,353.707500,64.675118,2.089720,712.637346,66.764838
300,2023-10-28,284.911936,67.054483,3.202960,620.954273,70.257443
301,2023-10-29,239.709040,50.032991,0.996537,593.167812,51.029528
302,2023-10-30,301.616857,68.187846,4.255661,690.266326,72.443507


In [103]:
df_all_all = df_all.merge(df2, on="Date")
df_all_all.set_index("Date", inplace=True)
df_all_all.to_pickle("data/all_historic_data.pkl")
df_all_all

,Wind,Max_tmp,Min_tmp,Mean_tmp,Rain,Sun,presMax,presMin,Eólica,Solar fotovoltaica,Solar térmica,Generación total,Solar
Date,,,,,,,,,,,,,
2023-01-01,2.800000,17.416667,7.566667,12.500000,4.383333,2.916667,1009.500000,1003.866667,160.764699,33.392982,0.811405,524.323313,34.204387
2023-01-02,2.366667,14.450000,6.916667,10.666667,0.816667,2.883333,1013.050000,1006.383333,96.936318,30.595371,0.194716,632.047278,30.790087
2023-01-03,2.433333,15.683333,5.383333,10.533333,0.000000,6.033333,1017.916667,1012.700000,69.070871,52.667047,3.462572,667.943963,56.129619
2023-01-04,2.266667,16.366667,6.016667,11.183333,0.000000,6.066667,1019.733333,1017.050000,65.706889,56.871036,4.732083,632.489265,61.603119
2023-01-05,2.083333,15.900000,4.750000,10.316667,0.000000,7.416667,1017.350000,1011.383333,43.799077,56.868750,5.249039,635.988081,62.117789
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-27,4.850000,21.116667,12.466667,16.783333,0.450000,4.516667,996.283333,992.183333,353.707500,64.675118,2.089720,712.637346,66.764838
2023-10-28,4.916667,21.150000,13.500000,17.316667,2.850000,5.533333,995.216667,990.600000,284.911936,67.054483,3.202960,620.954273,70.257443
2023-10-29,4.400000,20.916667,12.816667,16.866667,5.066667,3.750000,992.350000,988.483333,239.709040,50.032991,0.996537,593.167812,51.029528


In [65]:
historic_display = df_all_all[["Solar", "Eólica"]]
historic_display = historic_display.rename(columns={"Solar": "Solar", "Eólica": "Wind"})
historic_display = historic_display.reset_index()
historic_display.to_json("data/historic.json", date_format='iso', orient='split')

historic_display

,Date,Solar,Wind
0,2023-01-01,34.204387,160.764699
1,2023-01-02,30.790087,96.936318
2,2023-01-03,56.129619,69.070871
3,2023-01-04,61.603119,65.706889
4,2023-01-05,62.117789,43.799077
...,...,...,...
299,2023-10-27,66.764838,353.707500
300,2023-10-28,70.257443,284.911936
301,2023-10-29,51.029528,239.709040
302,2023-10-30,72.443507,301.616857


# Predictions

In [104]:
names = ["Barcelona", "Madrid", "Sevilla", "Bilbao", "A coruna", "Murcia"]
api_key = os.environ.get("API_worldweatheronline")

for name in names:
    url = f"https://api.worldweatheronline.com/premium/v1/weather.ashx?key={api_key}&q={name.lower()}&format=json&num_of_days=7"

    resp = requests.get(url)
    resp = resp.json()

    lst = []
    for day in resp["data"]["weather"]:

        mean_wind = 0
        for hour in day["hourly"]:
            mean_wind += float(hour["windspeedKmph"])
        mean_wind = mean_wind / len(day["hourly"])

        new_row = {
            "Date": day.get("date"),
            "City": name,
            "Wind": mean_wind,
            "Mean_tmp": day.get("avgtempC"),
            "Sun": day.get("sunHour"),

            # "Rain": day.get("prec"), 
            # "presMax": day.get("presMax"),
            # "Max_tmp": day.get("tmax"),
            # "Min_tmp": day.get("tmin"),
            # "presMin": day.get("presMin")
        }
        lst.append(new_row)

    df = pd.DataFrame(lst)

    df["Wind"] = df["Wind"].astype(float)
    df["Mean_tmp"] = df["Mean_tmp"].str.replace(',', '.').astype(float)
    df["Sun"] = df["Sun"].str.replace(',', '.').astype(float)
    df["Date"] = pd.to_datetime(df["Date"])
    df.to_pickle("data/" + name + "_pred.pkl")

# Add it all togeather

In [105]:
for name in names:
    df = pd.read_pickle(name + "_pred.pkl")
    df = df[["Date", "City", "Wind", "Mean_tmp", "Sun"]]
    df["City"] = name
    if name == "Barcelona":
        df_wide = df
    else:
        df_wide = pd.concat([df_wide, df])

df_all = df_wide[["Date", "Wind", "Mean_tmp", "Sun"]].groupby("Date").mean()
# df_wide
df_all.to_pickle("data/all_predictions.pkl")
df_all

,Wind,Mean_tmp,Sun
Date,,,
2023-11-23,10.979167,11.500000,9.550000
2023-11-24,8.250000,11.833333,9.100000
2023-11-25,6.500000,11.166667,9.016667
2023-11-26,5.291667,10.833333,9.050000
2023-11-27,8.604167,11.833333,8.783333
2023-11-28,9.541667,12.333333,2.933333
2023-11-29,10.312500,13.333333,5.116667
